In [18]:
import featuretools as tf
import numpy as np
import pandas as pd

train = pd.read_csv("/tmp/allInOne.csv")
print(train)

      netMosAvg  d2cLossAvg  e2eJitterAvg  d2cJitterAvg  e2eLossAvg  \
0       3.97741           4           125           125           4   
1       3.31220          17           286           286          17   
2       2.18346          31           492           492          31   
3       1.06915          48           723           723          48   
4       1.00000          61           958           958          61   
5       1.00000          78          1025          1025          78   
6       1.00000          88           965           965          88   
7       3.62769          12           146           146          12   
8       2.87999          23           366           366          23   
9       1.74661          35           562           562          35   
10      1.02937          52           841           841          52   
11      1.00000          65           981           981          65   
12      1.00000          82           972           972          82   
13    

In [19]:
train.isnull().sum()

netMosAvg       0
d2cLossAvg      0
e2eJitterAvg    0
d2cJitterAvg    0
e2eLossAvg      0
jitDepthAvg     0
playSpeedAvg    0
dtype: int64

In [20]:
# imputing missing data 
train['netMosAvg'].fillna(train['netMosAvg'].mean(), 
                            inplace = True) 

In [8]:
train['netMosAvg'].value_counts()

1.000000    1323
4.130000      31
2.940820      14
1.626000      12
2.753290      10
3.218070      10
2.925380      10
1.782500       8
3.200430       6
2.770930       6
2.925400       6
2.909970       6
2.788570       6
2.956250       4
3.086670       4
2.735670       4
1.043280       4
2.940420       4
1.015400       4
0.999147       4
2.565000       4
1.005230       4
3.182790       4
1.016240       4
0.000000       4
3.680740       3
3.612070       3
3.885700       3
3.927170       3
3.715460       3
            ... 
3.872530       1
3.741000       1
4.085920       1
3.954360       1
3.886230       1
3.631100       1
3.925480       1
3.894080       1
3.874720       1
3.894830       1
3.934780       1
3.868830       1
3.900220       1
4.067470       1
3.618670       1
3.929560       1
4.077670       1
3.805270       1
3.997620       1
3.932280       1
3.925790       1
4.090180       1
3.899950       1
3.985880       1
3.928010       1
3.925470       1
3.887030       1
3.774040      

In [21]:
list = []
for i in range (0,6084):
    list.append(i)
train.insert(0,'index',list)
print(train)

      index  netMosAvg  d2cLossAvg  e2eJitterAvg  d2cJitterAvg  e2eLossAvg  \
0         0    3.97741           4           125           125           4   
1         1    3.31220          17           286           286          17   
2         2    2.18346          31           492           492          31   
3         3    1.06915          48           723           723          48   
4         4    1.00000          61           958           958          61   
5         5    1.00000          78          1025          1025          78   
6         6    1.00000          88           965           965          88   
7         7    3.62769          12           146           146          12   
8         8    2.87999          23           366           366          23   
9         9    1.74661          35           562           562          35   
10       10    1.02937          52           841           841          52   
11       11    1.00000          65           981           981  

In [23]:
#creating and entity set 'es'
es = tf.EntitySet(id = 'netMosAvg')
#adding a dataframe
es.entity_from_dataframe(entity_id = 'bigmos', dataframe = train, index = "id")

2019-03-07 17:22:30,172 featuretools.entityset - WARNING    index id not found in dataframe, creating new integer column


Entityset: netMosAvg
  Entities:
    bigmos [Rows: 6084, Columns: 9]
  Relationships:
    No relationships

In [25]:
es.normalize_entity(base_entity_id='bigmos', 
                    new_entity_id='outmos', 
                    index = 'index', 
                    additional_variables =   
                    ['d2cLossAvg', 'e2eJitterAvg'])

Entityset: netMosAvg
  Entities:
    bigmos [Rows: 6084, Columns: 7]
    outmos [Rows: 6084, Columns: 3]
  Relationships:
    bigmos.index -> outmos.index

In [28]:
feature_matrix, feature_names = tf.dfs(entityset=es, 
                                       target_entity = 'bigmos', 
                                       max_depth = 2, 
                                       verbose = 1, 
                                       n_jobs = 3)
feature_matrix.columns

Built 39 features


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:51573 remote=tcp://127.0.0.1:51561>


EntitySet scattered to workers in 1.767 seconds
Elapsed: 00:04 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks

Future exception was never retrieved
future: <Future finished exception=CommClosedError('in <closed TCP>: Stream is closed')>
Traceback (most recent call last):
  File "/Users/wme/anaconda3/lib/python3.7/site-packages/distributed/comm/tcp.py", line 180, in read
    n_frames = yield stream.read_bytes(8)
  File "/Users/wme/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/wme/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/Users/wme/anaconda3/lib/python3.7/site-packages/distributed/comm/tcp.py", line 201, in read
    convert_stream_closed_error(self, e)
  File "/Users/wme/anaconda3/lib/python3.7/site-packages/distributed/comm/tcp.py", line 129, in convert_stream_closed_error
    raise CommClosed

Index(['index', 'netMosAvg', 'd2cJitterAvg', 'e2eLossAvg', 'jitDepthAvg',
       'playSpeedAvg', 'outmos.d2cLossAvg', 'outmos.e2eJitterAvg',
       'outmos.SUM(bigmos.netMosAvg)', 'outmos.SUM(bigmos.d2cJitterAvg)',
       'outmos.SUM(bigmos.e2eLossAvg)', 'outmos.SUM(bigmos.jitDepthAvg)',
       'outmos.SUM(bigmos.playSpeedAvg)', 'outmos.STD(bigmos.netMosAvg)',
       'outmos.STD(bigmos.d2cJitterAvg)', 'outmos.STD(bigmos.e2eLossAvg)',
       'outmos.STD(bigmos.jitDepthAvg)', 'outmos.STD(bigmos.playSpeedAvg)',
       'outmos.MAX(bigmos.netMosAvg)', 'outmos.MAX(bigmos.d2cJitterAvg)',
       'outmos.MAX(bigmos.e2eLossAvg)', 'outmos.MAX(bigmos.jitDepthAvg)',
       'outmos.MAX(bigmos.playSpeedAvg)', 'outmos.SKEW(bigmos.netMosAvg)',
       'outmos.SKEW(bigmos.d2cJitterAvg)', 'outmos.SKEW(bigmos.e2eLossAvg)',
       'outmos.SKEW(bigmos.jitDepthAvg)', 'outmos.SKEW(bigmos.playSpeedAvg)',
       'outmos.MIN(bigmos.netMosAvg)', 'outmos.MIN(bigmos.d2cJitterAvg)',
       'outmos.MIN(bigmos.e2eLossA

In [29]:
feature_matrix.head()

,index,netMosAvg,d2cJitterAvg,e2eLossAvg,jitDepthAvg,playSpeedAvg,outmos.d2cLossAvg,outmos.e2eJitterAvg,outmos.SUM(bigmos.netMosAvg),outmos.SUM(bigmos.d2cJitterAvg),...,outmos.MIN(bigmos.d2cJitterAvg),outmos.MIN(bigmos.e2eLossAvg),outmos.MIN(bigmos.jitDepthAvg),outmos.MIN(bigmos.playSpeedAvg),outmos.MEAN(bigmos.netMosAvg),outmos.MEAN(bigmos.d2cJitterAvg),outmos.MEAN(bigmos.e2eLossAvg),outmos.MEAN(bigmos.jitDepthAvg),outmos.MEAN(bigmos.playSpeedAvg),outmos.COUNT(bigmos)
id,,,,,,,,,,,,,,,,,,,,,
0,0,3.97741,125,4,534,1.2,4,125,3.97741,125,...,125,4,534,1.2,3.97741,125,4,534,1.2,1
1,1,3.31220,286,17,1186,1.0,17,286,3.31220,286,...,286,17,1186,1.0,3.31220,286,17,1186,1.0,1
2,2,2.18346,492,31,2038,1.0,31,492,2.18346,492,...,492,31,2038,1.0,2.18346,492,31,2038,1.0,1
3,3,1.06915,723,48,3032,1.0,48,723,1.06915,723,...,723,48,3032,1.0,1.06915,723,48,3032,1.0,1
4,4,1.00000,958,61,4046,1.0,61,958,1.00000,958,...,958,61,4046,1.0,1.00000,958,61,4046,1.0,1


In [32]:
feature_matrix = feature_matrix.reindex(index=train['id']) 
feature_matrix = feature_matrix.reset_index()

In [35]:
from catboost import CatBoostRegressor

In [36]:
#to string
categorical_features = np.where(feature_matrix.dtypes =='object')[0] 
for i in categorical_features: 
    feature_matrix.iloc[:,i]=feature_matrix.iloc[:,i].astype('str')

In [40]:
feature_matrix.drop(['id'], axis=1, inplace=True) 
train = feature_matrix[:5000] 
test = feature_matrix[5000:]
# removing uneccesary variables 
train.drop(['index'], axis=1, inplace=True) 
test.drop(['index'], axis=1, inplace=True)
# identifying categorical features categorical_features = np.where(train.dtypes == 'object')[0]

In [43]:
from sklearn.model_selection import train_test_split 
# splitting train data into training and validation set 
mos = train['netMosAvg']
xtrain, xvalid, ytrain, yvalid = train_test_split(train, mos, 
                                                  test_size=0.25, 
                                                  random_state=11)

In [49]:
model_cat = CatBoostRegressor(iterations=100, learning_rate=0.3, 
                              depth=6, eval_metric='RMSE',  
                              random_seed=7) 
# training model 
model_cat.fit(xtrain, ytrain, cat_features=categorical_features, 
              use_best_model=True)
# validation score 
model_cat.score(xvalid, yvalid)

You should provide test set for use best model. use_best_model parameter swiched to false value.


0:	learn: 1.8664825	total: 9.28ms	remaining: 919ms
1:	learn: 1.3373444	total: 14.8ms	remaining: 727ms
2:	learn: 0.9478320	total: 21.3ms	remaining: 689ms
3:	learn: 0.6797108	total: 27.5ms	remaining: 661ms
4:	learn: 0.4870956	total: 32.4ms	remaining: 615ms
5:	learn: 0.3538740	total: 35.4ms	remaining: 554ms
6:	learn: 0.2624788	total: 43.4ms	remaining: 577ms
7:	learn: 0.2017303	total: 51.3ms	remaining: 590ms
8:	learn: 0.1612876	total: 59.2ms	remaining: 599ms
9:	learn: 0.1364432	total: 64.4ms	remaining: 580ms
10:	learn: 0.1186946	total: 72.4ms	remaining: 586ms
11:	learn: 0.1052345	total: 79.7ms	remaining: 585ms
12:	learn: 0.0930063	total: 87.2ms	remaining: 583ms
13:	learn: 0.0880966	total: 94.5ms	remaining: 581ms
14:	learn: 0.0803138	total: 102ms	remaining: 578ms
15:	learn: 0.0772897	total: 106ms	remaining: 556ms
16:	learn: 0.0747361	total: 113ms	remaining: 552ms
17:	learn: 0.0692805	total: 118ms	remaining: 538ms
18:	learn: 0.0643309	total: 126ms	remaining: 538ms
19:	learn: 0.0639122	total:

0.02956417685808922